### MAC Season 6 Weather Data
- This notebook contains the code used to process raw weather station data from Maricopa Agricultural Center
- Data modified or added
    - sliced for season 6 dates only
        - planting: 2018-04-25
        - harvest: 2018-08-01
    - added dates
    - added cumulative growing degree days
    - added cumulative precipitation
    - changed name of `vpd_mean` columns to include units `vpd_mean_kpa`

In [ ]:
import datetime
import numpy as np
import pandas as pd

In [ ]:
# Read in raw season 6 weather data from MAC weather station

weather_df_0 = pd.read_csv('data/raw/mac/mac_weather_station_raw_daily_2018.csv')
print(weather_df_0.shape)
# weather_df_0.head(3)

#### Slice dataframe for season dates only and add date column

- Planting Date: 2018-04-25, Day of Year 115
- Last Day of Harvest: 2018-08-01, Day of Year 213

In [ ]:
weather_df_1 = weather_df_0.loc[(weather_df_0.day_of_year >= 115) & (weather_df_0.day_of_year <= 213)]
# print(weather_df_1.shape)
# weather_df_1.head(3)

In [ ]:
season_6_date_range = pd.date_range(start='2018-04-25', end='2018-08-01')

In [ ]:
weather_df_2 = weather_df_1.copy()
weather_df_2['date'] = season_6_date_range
# print(weather_df_2.shape)
# weather_df_2.tail(3)

#### Add Growing Degree Days
- 10 degrees Celsius is base temp for sorghum
- Daily gdd value = ((max temp + min temp) / 2) - 10 (base temp)
- Growing Degree Days = cumulative sum of daily gdd values
- Negative values convert to 0

In [ ]:
weather_df_3 = weather_df_2.copy()
weather_df_3['daily_gdd'] = (((weather_df_3['air_temp_max'] + weather_df_3['air_temp_min'])) / 2) - 10
# print(weather_df_3.shape)
# weather_df_3.head(3)

In [ ]:
# Check for negative values - will return empty df if there are none

weather_df_3.loc[weather_df_3.daily_gdd < 0]

In [ ]:
weather_df_4 = weather_df_3.copy()
weather_df_4['gdd'] = np.rint(np.cumsum(weather_df_4['daily_gdd']))
# print(weather_df_4.shape)
# weather_df_4.tail(3)

In [ ]:
# Drop daily gdd value, keep only cumulative gdd values

weather_df_5 = weather_df_4.drop(labels='daily_gdd', axis=1)
# print(weather_df_5.shape)
# weather_df_5.tail()

In [ ]:
weather_df_6 = weather_df_5.copy()
weather_df_6['cum_precip'] = np.round(np.cumsum(weather_df_6.precip_total), 2)
# weather_df_6.head(3)

#### Change name of vapor pressure column to include units

In [ ]:
weather_df_7 = weather_df_6.rename({'vpd_mean': 'vpd_mean_kpa'}, axis=1)
print(weather_df_7.shape)
weather_df_7.sample(n=3)

#### Write weather data to csv

In [ ]:
weather_df_7.to_csv('data/weather/mac_season_6_weather.csv', index=False)